In [1]:
import wandb

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random


from sklearn.model_selection import train_test_split

from catboost import CatBoostClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [ ]:
# connect wandb with your key

# wandb.login(api_key="YOUR_API_KEY")

# wandb login e7af045db59ef2d253743f547f333040a785e5ce
# OR
# wandb.login(api_key="e7af045db59ef2d253743f547f333040a785e5ce")

In [2]:
dtype={'id': np.dtype(int),
    'click': np.dtype(int),
    'hour': np.dtype(str),
    'C1': np.dtype(str),
    'banner_pos': np.dtype(str),
    'site_id': np.dtype(str),
    'site_domain': np.dtype(str), 
    'site_category': np.dtype(str),
    'app_id': np.dtype(str),
    'app_domain': np.dtype(str),
    'app_category': np.dtype(str),
    'device_id': np.dtype(str),
    'device_ip': np.dtype(str),
    'device_model': np.dtype(str),
    'device_type': np.dtype(str),
    'device_conn_type': np.dtype(str),
    'C14': np.dtype(str),
    'C15': np.dtype(str),
    'C16': np.dtype(str),
    'C17': np.dtype(str),
    'C18': np.dtype(str),
    'C19': np.dtype(str),
    'C20': np.dtype(str),
    'C21':np.dtype(str),
    'hour_of_day': np.dtype(str),
    'day_of_week': np.dtype(str),
      }#

In [3]:
df = pd.read_csv('/Users/fatmaoztel/Desktop/avazu-ctr-prediction/adjusted_dataset.csv',dtype=dtype,)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                10000 non-null  uint64
 1   click             10000 non-null  int64 
 2   hour              10000 non-null  object
 3   C1                10000 non-null  object
 4   banner_pos        10000 non-null  object
 5   site_id           10000 non-null  object
 6   site_domain       10000 non-null  object
 7   site_category     10000 non-null  object
 8   app_id            10000 non-null  object
 9   app_domain        10000 non-null  object
 10  app_category      10000 non-null  object
 11  device_id         10000 non-null  object
 12  device_ip         10000 non-null  object
 13  device_model      10000 non-null  object
 14  device_type       10000 non-null  object
 15  device_conn_type  10000 non-null  object
 16  C14               10000 non-null  object
 17  C15          

In [5]:
df.drop(columns=["id", "hour"], inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   click             10000 non-null  int64 
 1   C1                10000 non-null  object
 2   banner_pos        10000 non-null  object
 3   site_id           10000 non-null  object
 4   site_domain       10000 non-null  object
 5   site_category     10000 non-null  object
 6   app_id            10000 non-null  object
 7   app_domain        10000 non-null  object
 8   app_category      10000 non-null  object
 9   device_id         10000 non-null  object
 10  device_ip         10000 non-null  object
 11  device_model      10000 non-null  object
 12  device_type       10000 non-null  object
 13  device_conn_type  10000 non-null  object
 14  C14               10000 non-null  object
 15  C15               10000 non-null  object
 16  C16               10000 non-null  object
 17  C17          

In [7]:
X=df.drop(['click'],axis=1)
y=df['click']
X

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,...,C14,C15,C16,C17,C18,C19,C20,C21,hour_of_day,day_of_week
0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,431b3174,...,15704,320,50,1722,0,35,-1,79,0,Tuesday
1,1005,0,3695113d,8a9459c0,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,e7ab261c,...,20366,320,50,2333,0,39,100103,157,0,Tuesday
2,1005,0,543a539e,c7ca3108,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,d0aa00c7,...,20352,320,50,2333,0,39,-1,157,0,Tuesday
3,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ef85aaad,...,15705,320,50,1722,0,35,100084,79,0,Tuesday
4,1005,0,85f751fd,c4e18dd6,50e219e0,1779deee,2347f47a,f95efa07,a99f214a,ba92cd0f,...,20596,320,50,2161,0,35,-1,157,0,Tuesday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,a99f214a,8f642135,...,21611,320,50,2480,3,299,100111,61,23,Thursday
9996,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,a99f214a,d30e5a48,...,23161,320,50,2667,0,47,-1,221,23,Thursday
9997,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,a99f214a,9cf8ab70,...,23160,320,50,2667,0,47,-1,221,23,Thursday
9998,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,10dcdfb1,...,23723,320,50,2716,3,47,-1,23,23,Thursday


Accuracy is checked on the validation dataset, which has data in the same format as in the training dataset, but it is only used for evaluating the quality of training (it is not used for training).

CatBoost is based on gradient boosted decision trees. During training, a set of decision trees is built consecutively. Each successive tree is built with reduced loss compared to the previous trees.*

- *https://catboost.ai/en/docs/concepts/algorithm-main-stages*

In [8]:

# X: Features, y: Target variable
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42) # 80% train
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # 10% test, 10% valid


While the 80-10-10 split is common, it's not set in stone. For smaller datasets, you might want to allocate a larger portion to training. In some cases, you might even consider a 70-15-15 split or variations depending on your specific needs and constraints.

In [9]:
X_train

,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,...,C14,C15,C16,C17,C18,C19,C20,C21,hour_of_day,day_of_week
9254,1005,0,85f751fd,c4e18dd6,50e219e0,1cc958a2,2347f47a,f95efa07,a99f214a,2022a4c8,...,23804,320,50,2726,3,803,100148,229,5,Thursday
1561,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,a4afe726,...,15702,320,50,1722,0,35,-1,79,10,Wednesday
1670,1005,0,85f751fd,c4e18dd6,50e219e0,e9739828,df32afa9,cef3e649,a99f214a,fe75d0c7,...,21769,320,50,2507,0,35,-1,157,11,Wednesday
6087,1005,0,93eaba74,7687a86e,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,26b8bb1b,...,17654,300,250,1994,2,39,-1,33,6,Monday
6669,1005,1,856e6d3f,58a89a43,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,ab39bdf7,...,19772,320,50,2227,0,935,-1,48,22,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,1005,0,85f751fd,c4e18dd6,50e219e0,090d3a47,0654b444,0f2161f8,a99f214a,b4b35bee,...,17212,320,50,1887,3,39,100194,23,17,Sunday
5191,1005,1,178a7b89,b4598159,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,7ab43b94,...,19771,320,50,2227,0,679,100081,48,7,Sunday
5390,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,0fa21480,...,15706,320,50,1722,0,35,-1,79,11,Sunday
860,1005,0,85f751fd,c4e18dd6,50e219e0,2d869bee,d9b5648e,0f2161f8,a99f214a,1ca2d148,...,20751,320,50,1895,0,681,100028,101,19,Tuesday


In [10]:
y_train

9254    0
1561    1
1670    0
6087    0
6669    0
       ..
5734    0
5191    0
5390    0
860     0
7270    0
Name: click, Length: 8000, dtype: int64

In [11]:
cat_features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,21,22]

In [20]:
model = CatBoostClassifier(
    iterations=50,
    learning_rate=0.5,
    task_type='CPU', # also GPU is available
    loss_function='Logloss',
    gpu_ram_part=0.9,
    boosting_type='Plain',
    max_ctr_complexity=2,
    depth=6,
    gpu_cat_features_storage='CpuPinnedMemory',
)

In [31]:
cat_features = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,21,22]

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid), (X_test, y_test)],
    cat_features=cat_features,
    verbose=10
)

Learning rate set to 0.052963
0:	learn: 0.6663901	test: 0.6658557	test1: 0.6667779	best: 0.6667779 (0)	total: 2.76ms	remaining: 2.76s
10:	learn: 0.5031690	test: 0.4986144	test1: 0.5019958	best: 0.5019958 (10)	total: 116ms	remaining: 10.4s
20:	learn: 0.4535274	test: 0.4467004	test1: 0.4514476	best: 0.4514476 (20)	total: 221ms	remaining: 10.3s
30:	learn: 0.4356049	test: 0.4285693	test1: 0.4331824	best: 0.4331824 (30)	total: 322ms	remaining: 10.1s
40:	learn: 0.4268003	test: 0.4211129	test1: 0.4252696	best: 0.4252696 (40)	total: 429ms	remaining: 10s
50:	learn: 0.4209515	test: 0.4169415	test1: 0.4206717	best: 0.4206717 (50)	total: 546ms	remaining: 10.2s
60:	learn: 0.4168629	test: 0.4155252	test1: 0.4176054	best: 0.4176054 (60)	total: 685ms	remaining: 10.6s
70:	learn: 0.4128312	test: 0.4153054	test1: 0.4161585	best: 0.4161585 (70)	total: 846ms	remaining: 11.1s
80:	learn: 0.4105385	test: 0.4154538	test1: 0.4153505	best: 0.4153505 (80)	total: 984ms	remaining: 11.2s
90:	learn: 0.4086178	test: 0

In [33]:
model.get_feature_importance(prettified=True)


,Feature Id,Importances
0,site_domain,6.682525
1,app_id,6.675858
2,C21,6.593546
3,site_id,6.551978
4,device_model,6.247201
5,day_of_week,6.028937
6,hour_of_day,6.015655
7,C17,6.013983
8,C20,5.764570
9,C19,5.374555


In [12]:
import wandb



CatBoost's integration with Bayesian optimization can help you find good hyperparameter settings in fewer iterations compared to grid search or random search. It takes into account the information from previous iterations to guide the search towards areas that are likely to lead to better performance.

no space to wrtie in visual studio

In [12]:
# Define the sweep configuration
sweep_config = {
    "method": "bayes",
    
    'metric': {
      'name': 'average_train_loss',
      'goal': 'minimize'   
    },
    
    "metric": {
        "name": "average_valid_loss",
        "goal": "minimize"
    },
    "parameters": {
        "learning_rate": {"min": 0.01, "max": 0.2},
        "depth": {"min": 4, "max": 10},
        "iterations": {"min": 100, "max": 300}
    }
}

# Create the sweep
sweep_id = wandb.sweep(sweep_config, project="ctr-prediction")

# Define the train function
def train():
    # Initialize wandb for the current run
    with wandb.init(config=wandb.config):
        # Fetch hyperparameters from the wandb.config object
        learning_rate = wandb.config.learning_rate
        depth = wandb.config.depth
        iterations = wandb.config.iterations

        # Create CatBoost model with current hyperparameters
        model = CatBoostClassifier(
            learning_rate=learning_rate,
            depth=depth,
            iterations=iterations,
            cat_features=cat_features,  # Add your categorical feature indices here
            early_stopping_rounds=10,  # Stop if the validation loss doesn't improve for 10 rounds
            use_best_model=True,  # Enable early stopping using the best model
            verbose=0
        )

        # Train the model
        model.fit(X_train, y_train, eval_set=(X_valid, y_valid))

        # Calculate average training loss
        train_preds = model.predict_proba(X_train)
        average_train_loss = log_loss(y_train, train_preds) #cross-entropy loss

        # Calculate average validation loss
        valid_preds = model.predict_proba(X_valid)
        average_valid_loss = log_loss(y_valid, valid_preds) #cross-entropy loss

        # Log average losses in W&B
        wandb.log({"average_train_loss": average_train_loss, "average_valid_loss": average_valid_loss})

        # Make predictions on the validation set
        y_valid_pred = model.predict(X_valid)

        # Calculate and log accuracy, precision, recall, and F1-score for validation set
        accuracy = accuracy_score(y_valid, y_valid_pred)
        precision = precision_score(y_valid, y_valid_pred)
        recall = recall_score(y_valid, y_valid_pred)
        f1 = f1_score(y_valid, y_valid_pred)

        wandb.log({"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1})

# Run the sweep
wandb.agent(sweep_id, train)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: itizfcqu
Sweep URL: https://wandb.ai/ftmoztl/ctr-prediction/sweeps/itizfcqu


wandb: Agent Starting Run: ld75etzs with config:
wandb: 	depth: 8
wandb: 	iterations: 300
wandb: 	learning_rate: 0.1207369724601728
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ftmoztl. Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy,▁
average_train_loss,▁
average_valid_loss,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.834
average_train_loss,0.3928
average_valid_loss,0.41215
f1_score,0.07778
precision,0.4375


...

accuracy,▁
average_train_loss,▁
average_valid_loss,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.833
average_train_loss,0.38916
average_valid_loss,0.4115
f1_score,0.10695
precision,0.43478


wandb: Agent Starting Run: qbssf9rg with config:
wandb: 	depth: 5
wandb: 	iterations: 280
wandb: 	learning_rate: 0.07660584739954515
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


accuracy,▁
average_train_loss,▁
average_valid_loss,▁
f1_score,▁
precision,▁
recall,▁
accuracy,0.834
average_train_loss,0.39606
average_valid_loss,0.41479
f1_score,0.02353
precision,0.33333


wandb: Ctrl + C detected. Stopping sweep.


### Learning Rate Scheduler

Using a learning rate scheduler can be beneficial for training machine learning models, including CatBoost models, as it can help improve convergence and potentially lead to better performance. However, CatBoost already has a built-in learning rate reduction strategy based on early stopping, so you need to consider the trade-offs and your specific use case.



### Epochs or Iteration

In CatBoost, the concept of "epochs" isn't explicitly used in the same way it might be in some other machine learning algorithms like neural networks. CatBoost is a gradient boosting algorithm that builds an ensemble of decision trees iteratively. Instead of epochs, CatBoost uses the term "iterations" to refer to the number of boosting rounds during which new trees are added to the ensemble.

Each iteration in CatBoost corresponds to the creation of a new decision tree. The algorithm focuses on minimizing the loss function by iteratively adding trees that correct the mistakes of the previous ones. This process continues until the specified number of iterations is reached or until early stopping criteria are met.

CatBoost has built-in mechanisms to handle early stopping, which allows the algorithm to automatically stop adding trees when the performance on the validation set starts to degrade. You can specify the number of iterations when creating a CatBoost model, and you can also set early stopping criteria such as the number of rounds without improvement in validation loss.

### Loss Function

The cross-entropy loss function is generally suitable for click-through rate (CTR) prediction tasks, including in the training section of CatBoost. The cross-entropy loss is a common choice for binary classification problems like CTR prediction, where the target variable has two classes (clicked or not clicked). And also, the cross-entropy loss is well-suited for imbalanced datasets as it penalizes misclassifying the minority class more heavily, helping the model focus on the class of interest. Our dataset is also imbalanced dataset.

Of course, there are other loss functions like (MSE) Loss, SVM Loss etc. 

MAPEeeeeeee